In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [32]:
spot = pd.read_csv(
    "data/processed/nifty_spot_clean_5min.csv",
    parse_dates=['date'],
    index_col='date'
)

futures = pd.read_csv(
    "data/processed/nifty_futures_clean_5min.csv",
    parse_dates=['date'],
    index_col='date'
)

options = pd.read_csv(
    "data/processed/nifty_options_clean_5min.csv",
    parse_dates=['date'],
    index_col='date'
)


In [33]:
# Extract ATM options

In [34]:
atm_opts = options[options['strike'] == options['spot_close'].round(-2)]

ce = atm_opts[atm_opts['option_type'] == 'CE']
pe = atm_opts[atm_opts['option_type'] == 'PE']


In [35]:
#constructing feature dataframe 

In [36]:
spot['log_return'] = np.log(spot['close']).diff()


In [37]:
# Recompute PCR (OI-based)
pcr = (
    options
    .groupby([options.index, 'option_type'])['open_interest']
    .sum()
    .unstack()
)

pcr['pcr_oi'] = pcr['PE'] / pcr['CE']


In [38]:
spot = spot.merge(
    pcr[['pcr_oi']],
    left_index=True,
    right_index=True,
    how='left'
)


In [39]:
spot['pcr_oi'].describe()


count    18676.000000
mean         1.000677
std          0.036606
min          0.867633
25%          0.975134
50%          0.999919
75%          1.025232
max          1.136153
Name: pcr_oi, dtype: float64

In [40]:
from scipy.stats import norm

T = 1 / 252   # 1 trading day to expiry
r = 0.06     # risk-free rate


In [41]:
def bs_d1(S, K, T, r, sigma):
    return (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))

def bs_d2(S, K, T, r, sigma):
    return bs_d1(S, K, T, r, sigma) - sigma * np.sqrt(T)

def delta(S, K, T, r, sigma, option_type):
    d1 = bs_d1(S, K, T, r, sigma)
    if option_type == 'CE':
        return norm.cdf(d1)
    else:
        return norm.cdf(d1) - 1

def gamma(S, K, T, r, sigma):
    d1 = bs_d1(S, K, T, r, sigma)
    return norm.pdf(d1) / (S * sigma * np.sqrt(T))

def vega(S, K, T, r, sigma):
    d1 = bs_d1(S, K, T, r, sigma)
    return S * norm.pdf(d1) * np.sqrt(T)


In [42]:
atm_opts = options[options['strike'] == options['spot_close'].round(-2)].copy()

atm_opts['delta'] = atm_opts.apply(
    lambda x: delta(
        x['spot_close'], x['strike'], T, r, x['iv'], x['option_type']
    ),
    axis=1
)

atm_opts['gamma'] = atm_opts.apply(
    lambda x: gamma(
        x['spot_close'], x['strike'], T, r, x['iv']
    ),
    axis=1
)

atm_opts['vega'] = atm_opts.apply(
    lambda x: vega(
        x['spot_close'], x['strike'], T, r, x['iv']
    ),
    axis=1
)


In [43]:
ce = atm_opts[atm_opts['option_type'] == 'CE']
pe = atm_opts[atm_opts['option_type'] == 'PE']


In [44]:
hmm_features['atm_delta'] = (
    ce['delta'].abs().values + pe['delta'].abs().values
) / 2

hmm_features['atm_gamma'] = ce['gamma'].values
hmm_features['atm_vega'] = ce['vega'].values


ValueError: Length of values (18676) does not match length of index (18675)

In [ ]:
hmm_features = pd.DataFrame(index=spot.index)

hmm_features['spot_return'] = spot['log_return']
hmm_features['futures_basis'] = futures['close'] - spot['close']
hmm_features['pcr'] = spot['pcr_oi']
# IV features
hmm_features['avg_iv'] = (
    ce['iv'].add(pe['iv']).div(2)
)

hmm_features['iv_spread'] = ce['iv'] - pe['iv']

hmm_features['atm_delta'] = (
    ce['delta'].abs().add(pe['delta'].abs()).div(2)
)

hmm_features['atm_gamma'] = ce['gamma']
hmm_features['atm_vega'] = ce['vega']


hmm_features = hmm_features.dropna()


In [ ]:
hmm_features.shape



In [ ]:
hmm_features.head()

In [ ]:
hmm_features.isna().sum()


In [ ]:
split_idx = int(len(hmm_features) * 0.7)

train_idx = hmm_features.index[:split_idx]
test_idx = hmm_features.index[split_idx:]

train = hmm_features.loc[train_idx]
test = hmm_features.loc[test_idx]


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train = scaler.fit_transform(train)
X_test = scaler.transform(test)



In [ ]:
pip install hmmlearn


In [ ]:
from hmmlearn.hmm import GaussianHMM

hmm = GaussianHMM(
    n_components=3,
    covariance_type="full",
    n_iter=300,
    random_state=42
)

hmm.fit(X_train)


In [ ]:
train_states = hmm.predict(X_train)
test_states = hmm.predict(X_test)

hmm_features.loc[train_idx, 'regime_raw'] = train_states
hmm_features.loc[test_idx, 'regime_raw'] = test_states


In [ ]:
hmm_features['regime_raw'].value_counts()


In [ ]:
regime_returns = (
    hmm_features
    .groupby('regime_raw')['spot_return']
    .mean()
    .sort_values()
)

regime_map = {
    regime_returns.index[0]: -1,  # Downtrend
    regime_returns.index[1]: 0,   # Sideways
    regime_returns.index[2]: 1    # Uptrend
}

hmm_features['regime'] = hmm_features['regime_raw'].map(regime_map)


In [ ]:
hmm_features['regime'].value_counts()


In [ ]:
spot = spot.merge(
    hmm_features[['regime']],
    left_index=True,
    right_index=True,
    how='left'
)


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12,5))
plt.plot(spot['close'], color='black', linewidth=1)

plt.scatter(
    spot.index,
    spot['close'],
    c=spot['regime'],
    cmap='RdYlGn',
    s=4
)

plt.title("NIFTY Intraday Price with HMM Regimes")
plt.show()


In [ ]:
import seaborn as sns

sns.heatmap(hmm.transmat_, annot=True, cmap='Blues')
plt.title("HMM Regime Transition Matrix")
plt.show()


In [ ]:
hmm_features.groupby('regime')[[
    'avg_iv','iv_spread','pcr',
    'atm_delta','atm_gamma','atm_vega'
]].mean()


In [ ]:
regime_blocks = (
    hmm_features['regime']
    .ne(hmm_features['regime'].shift())
    .cumsum()
)

durations = regime_blocks.value_counts()

plt.hist(durations, bins=30)
plt.title("Regime Duration Distribution")
plt.show()


In [ ]:
#merging regime shorter than 3 bars

In [ ]:
min_duration = 3

regime_series = hmm_features['regime'].copy()

groups = regime_series.ne(regime_series.shift()).cumsum()
durations = groups.value_counts()

short_groups = durations[durations < min_duration].index

for g in short_groups:
    idx = groups[groups == g].index
    prev_idx = regime_series.index.get_loc(idx[0]) - 1
    if prev_idx >= 0:
        regime_series.loc[idx] = regime_series.iloc[prev_idx]

hmm_features['regime_smoothed'] = regime_series

In [ ]:
hmm_features[['regime','regime_smoothed']].head(10)


In [ ]:
smoothed_groups = (
    hmm_features['regime_smoothed']
    .ne(hmm_features['regime_smoothed'].shift())
    .cumsum()
)

smoothed_durations = smoothed_groups.value_counts()


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8,4))
plt.hist(smoothed_durations, bins=30)
plt.title("Smoothed Regime Duration Distribution")
plt.xlabel("Bars per Regime")
plt.ylabel("Frequency")
plt.show()


In [ ]:
spot = spot.merge(
    hmm_features[['regime_smoothed']],
    left_index=True,
    right_index=True,
    how='left'
)

plt.figure(figsize=(12,5))
plt.plot(spot['close'], color='black', linewidth=1)

plt.scatter(
    spot.index,
    spot['close'],
    c=spot['regime_smoothed'],
    cmap='RdYlGn',
    s=4
)

plt.title("NIFTY Price with Smoothed HMM Regimes")
plt.show()


In [ ]:
hmm_features[['regime']].to_csv("data/processed/hmm_regimes.csv")

In [ ]:
spot['regime'].value_counts()
